# Create a new Java database and search model class

Specify the details of your new Java class below. 

In [ ]:
eval $(./vars.py)
CLASS_PACKAGE=$SITE_JAVA_PACKAGE.model.node
CLASS_SIMPLE_NAME=AiNode
CLASS_A_NAME="an AI node"
CLASS_DESCRIPTION="A Red Hat OpenShift node containing GPUs"
CLASS_ORDER=5
CLASS_ICON='<i class="fa-regular fa-computer"></i>'
echo DONE

## Create a Keycloak auth resource for your model


Create a Keycloak auth Resource for your new model in the site. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/resource" \
  --data '{"name":"'"$CLASS_SIMPLE_NAME"'","displayName":"'"$CLASS_SIMPLE_NAME"'","scopes":["POST","PATCH","GET","'"$AUTH_SCOPE_ADMIN"'","'"$AUTH_SCOPE_SUPER_ADMIN"'"]}'

echo DONE

## Grant POST, PATCH, GET, SiteAdmin permissions for SiteAdmins to your new model


Create a Keycloak auth Permission for SiteAdmins to use the POST, PATCH, GET and SiteAdmin scopes of your model. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/permission/scope" \
  --data '{"name":"group-'"$AUTH_SCOPE_ADMIN"'-resource-'"$CLASS_SIMPLE_NAME"'"
      ,"decisionStrategy":"AFFIRMATIVE"
      ,"description":"'"$AUTH_SCOPE_ADMIN"' group access to all '"$CLASS_SIMPLE_NAME"'"
      ,"resources":["'"$CLASS_SIMPLE_NAME"'"]
      ,"policies":["'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_ADMIN"'"]
      ,"scopes":["'"$AUTH_REALM"'-POST","'"$AUTH_REALM"'-PATCH","'"$AUTH_REALM"'-GET","'"$AUTH_REALM"'-'"$AUTH_SCOPE_ADMIN"'"]
      }'

echo DONE

## Grant POST, PATCH, GET, SiteAdmin, SuperAdmin permissions for SuperAdmins to your new model


Create a Keycloak auth Permission for SuperAdmins to use the POST, PATCH, GET, SiteAdmin, and SuperAdmin scopes of your model. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/permission/scope" \
  --data '{"name":"group-'"$AUTH_SCOPE_SUPER_ADMIN"'-resource-'"$CLASS_SIMPLE_NAME"'"
      ,"decisionStrategy":"AFFIRMATIVE"
      ,"description":"'"$AUTH_SCOPE_SUPER_ADMIN"' group access to all '"$CLASS_SIMPLE_NAME"'"
      ,"resources":["'"$CLASS_SIMPLE_NAME"'"]
      ,"policies":["'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_SUPER_ADMIN"'"]
      ,"scopes":["'"$AUTH_REALM"'-POST","'"$AUTH_REALM"'-PATCH","'"$AUTH_REALM"'-GET","'"$AUTH_REALM"'-'"$AUTH_SCOPE_ADMIN"'","'"$AUTH_REALM"'-'"$AUTH_SCOPE_SUPER_ADMIN"'"]
      }'

echo DONE

## Compile your Java project

In [ ]:
eval $(./vars.py)
(cd $SITE_SRC && mvn clean install)
echo DONE